In [ ]:
# persianpress.dh.huji.ac.il/issues api keys

KEY_IDENTITY = "1234578890abcede"  # Replace with your key identity
KEY_CREDENTIAL = "987654321polika"

In [ ]:
# check to see that you can connect to the api

In [ ]:
import requests

API_URL = "https://persianpress.dh.huji.ac.il/issues/api"

headers = {
        'Authorization': f'key_identity={KEY_IDENTITY},key_credential={KEY_CREDENTIAL}'
    }

response = requests.get(API_URL, headers=headers, verify=False)
if response.status_code == 200:
    print("Successfully connected to the API.")
    if response.text:
        print("Raw response:", response.text)

else:
    print("Error:", response.status_code)


In [ ]:
# explore api

In [ ]:
import requests

API_URL = "https://persianpress.dh.huji.ac.il/issues/api/resource_templates"

headers = {
        'Authorization': f'key_identity={KEY_IDENTITY},key_credential={KEY_CREDENTIAL}'
    }

response = requests.get(API_URL, headers=headers, verify=False)
if response.status_code == 200:
    print("Successfully connected to the API.")
    if response.text:
        print("Raw response:", response.text)

else:
    print("Error:", response.status_code)


In [ ]:
import requests

API_URL = "https://persianpress.dh.huji.ac.il/issues/api/items/1954"

headers = {
        'Authorization': f'key_identity={KEY_IDENTITY},key_credential={KEY_CREDENTIAL}'
    }

response = requests.get(API_URL, headers=headers, verify=False)
if response.status_code == 200:
    print("Successfully connected to the API.")
    if response.text:
        print("Raw response:", response.text)

else:
    print("Error:", response.status_code)


In [ ]:
results = response.json()
results

In [ ]:
response.json()['dcterms:identifier']

In [ ]:
#create new "page" item with jpg and xml as the media, and use metadata to fill in properties
import re
import json
from pathlib import Path
import requests 

# Sample complex path with folder structure
jpg_path = Path(r"C:\Users\User\PycharmProjects\PersianChildhood\Journals Data\Ḥabl al-matı̄n\1. Jahrgang (1325-1326hq -- 829hš -- 1907-1908)\2. Ausgabe (16. Rabīʿ al-awwal 1325 -- 10. Ordibehešt 829 -- 30. April 1907)\3.jpg")
xml_path = "3.xml"

API_URL = "https://persianpress.dh.huji.ac.il/issues/api/items"
#API_URL = "https://persianpress.dh.huji.ac.il/issues/api"

# Function to extract metadata from the full folder structure
def extract_metadata_from_path(file_path: Path):
    parts = file_path.parts

    # Attempt to extract title and issue from folder names
    try:
        journal_name = parts[-3]  # e.g., 'Ḥabl al-matı̄n'
        issue_folder = parts[-2]  # e.g., '2. Ausgabe (...)'
        page_number = file_path.stem  # e.g., '2'

        # Extract issue number and label
        issue_match = re.match(r"(\d+)\. Ausgabe.*", issue_folder)
        issue_number = issue_match.group(1) if issue_match else "?"

        return {
            "title": f"{journal_name} - Issue {issue_number} - Page {page_number}",
            "identifier": f"Issue_{issue_number}_Page_{page_number}",
            "issue": issue_folder,
            "page": page_number
        }
    except Exception as e:
        return {
            "title": "Untitled",
            "identifier": "unknown",
            "issue": "unknown",
            "page": "0",
            "error": str(e)
        }

# Extract text content from XML file
def extract_text_from_xml_safe(xml_file):
    xml_file = Path(xml_file)  # ⬅ This line converts string to Path
    if xml_file.exists():
        try:
            return xml_file.read_text(encoding="utf-8")
        except Exception as e:
            return f"[Error reading XML: {str(e)}]"
    return "[No XML file found]"


# Extract metadata
metadata = extract_metadata_from_path(jpg_path)
extracted_text = extract_text_from_xml_safe(xml_path)

# Build base JSON structure
item_data = {
    "o:resource_template": {"o:id": 2},
    "o:resource_class": {"o:id": 50},
    "o:is_public": True,
    "dcterms:title": [
        {"type": "literal", "property_id": 1, "property_label": "Title", "is_public": True, "@value": metadata["title"]}
    ],
    "dcterms:identifier": [
        {"type": "literal", "property_id": 10, "property_label": "Identifier", "is_public": True, "@value": metadata["identifier"]}
    ],
    "foaf:page": [
        {"type": "literal", "property_id": 174, "property_label": "page", "is_public": True, "@value": metadata["page"]}
    ],
    "bibo:issue": [
        {"type": "literal", "property_id": 103, "property_label": "issue", "is_public": True, "@value": metadata["issue"]}
    ],
    "dcterms:description": [
        {"type": "literal", "property_id": 4, "property_label": "Description", "is_public": True, "@value": "Created automatically via Omeka API"}
    ],
#     "extracttext:extracted_text": [
#         {"type": "literal", "property_id": 185, "property_label": "extracted text", "is_public": True, "@value": extracted_text}
#     ],
    "o:media": [
        {
            "o:ingester": "upload",
            "file_index": 0,
            "dcterms:title": [{"@value": "Scanned Page"}]
        },
        {
            "o:ingester": "upload",
            "file_index": 1,
            "dcterms:title": [{"@value": "XML OCR"}]
        }
    ]
}

# Display JSON structure
# json.dumps(item_data, indent=2, ensure_ascii=False)

# --- PREPARE MULTIPART FORM DATA ---
files = {
    'data': (None, json.dumps(item_data), 'application/json'),
    'file[0]': (Path(jpg_path).name, open(jpg_path, 'rb'), 'image/jpeg'),
    'file[1]': (Path(xml_path).name, open(xml_path, 'rb'), 'text/xml'),
}

params = {
    'key_identity': KEY_IDENTITY,
    'key_credential': KEY_CREDENTIAL
}

# --- SEND REQUEST ---
response = requests.post(API_URL, files=files, params=params, verify=False)

# --- HANDLE RESPONSE ---
if response.status_code in [200, 201]:
    print("✅ Item created successfully!")
    print(json.dumps(response.json(), indent=2))
else:
    print(f"❌ Error {response.status_code}:")
    print(response.text)


In [ ]:
# mass pipeline attempts

In [ ]:
# this is currently the working version
import json
import re
import requests
from pathlib import Path
import pandas as pd

# --- API Info ---
API_BASE = "https://persianpress.dh.huji.ac.il/issues/api"

params = {
    "key_identity": KEY_IDENTITY,
    "key_credential": KEY_CREDENTIAL
}
headers = {
    "Content-Type": "application/json"
}

# --- Template IDs ---
TEMPLATE_JOURNAL = 4
TEMPLATE_ISSUE = 3
TEMPLATE_PAGE = 2

# --- Class IDs ---
CLASS_JOURNAL = 61  # Bibliographic Resource
CLASS_ISSUE = 60    # Periodical Issue
CLASS_PAGE = 50     # Page

# --- Property IDs ---
PROPERTY_ID_TITLE = 1         # dcterms:title
PROPERTY_ID_IDENTIFIER = 10   # dcterms:identifier
PROPERTY_ID_DATE = 7          # dcterms:date
PROPERTY_ID_IS_PART_OF = 33   # dcterms:isPartOf

# === Load Metadata ===
metadf = pd.read_excel("Persian Literature Metadata.xlsx")
# Sample filter for testing:
test_df = metadf[metadf["journal"] == "Ḥabl al-matı̄n"]

# === Utility: check if item exists by identifier ===
def get_resource_by_identifier(identifier):
    """
    Return the first Omeka item ID that has dcterms:identifier == identifier,
    or None if not found.
    """
    search_params = {
        "key_identity": KEY_IDENTITY,
        "key_credential": KEY_CREDENTIAL,
        "property[0][property]": "dcterms:identifier",
        "property[0][type]": "eq",
        "property[0][text]": identifier
    }
    r = requests.get(f"{API_BASE}/items", params=search_params, verify=False)
    if r.status_code == 200:
        results = r.json()
        if results:
            found_id = results[0]["o:id"]
            print(f"   Found existing item [ID={found_id}] for identifier='{identifier}'")
            return found_id
    return None

# === Utility: read in the .xml OCR content if present ===
def read_xml_content(xml_path: Path):
    try:
        return xml_path.read_text(encoding="utf-8")
    except Exception as e:
        return f"[XML read error: {str(e)}]"

# === Utility: optionally parse a date from the issue label ===
def extract_date_from_issue_label(issue_label):
    """
    Example: parse something like '7. Rabīʿ 1326' from the issue string.
    Adjust or remove if not matching your data.
    """
    match = re.search(r'(\d{1,2}\.\s*\w+\s+\d{4})', issue_label)
    return match.group(1) if match else None

# --- Main Loop ---
for idx, row in test_df.iterrows():
    uid = str(row["unique_id"])
    journal = str(row["journal"])  # e.g. "Ḥabl al-matı̄n"
    volume = str(row["volume"])
    issue = str(row["issue"])
    page = str(row["page"])

    print("\n----------------------------------------------------")
    print(f"Processing row={idx}: Journal={journal}, Issue={issue}, Page={page}")

    # --- Build stable identifiers
    journal_identifier = journal                  # Must match EXACTLY
    issue_identifier = f"{journal} - Issue {issue}"
    page_identifier = uid
    
    # skip creating a page if it already exists
    existing_page_id = get_resource_by_identifier(page_identifier)
    if existing_page_id:
        print(f"   Page '{page_identifier}' already exists, ID={existing_page_id}. Skipping.")
        continue

    # -----------------------------------------------------------
    # 1) Ensure the JOURNAL exists (attempt to find it first)
    # -----------------------------------------------------------
    print(f"Checking if Journal '{journal}' already exists by identifier='{journal_identifier}'")
    journal_id = get_resource_by_identifier(journal_identifier)
    if journal_id:
        print(f"   ✅ Re-using existing Journal, ID={journal_id}")
    else:
        print(f"   ❌ Not found. Creating new Journal => '{journal}'")
        # Exactly your snippet's approach:
        journal_payload = {
            "o:resource_template": {"o:id": TEMPLATE_JOURNAL},
            "o:resource_class": {"o:id": CLASS_JOURNAL},
            "o:is_public": True,
            "dcterms:title": [{
                "type": "literal",
                "property_id": PROPERTY_ID_TITLE,
                "@value": journal
            }],
            "dcterms:identifier": [{
                "type": "literal",
                "property_id": PROPERTY_ID_IDENTIFIER,
                "@value": journal_identifier
            }]
        }
        journal_response = requests.post(
            f"{API_BASE}/items",
            params=params,
            headers=headers,
            json=journal_payload,
            verify=False
        )
        # if it worked we'll get a new id
        if journal_response.status_code in [200, 201]:
            journal_id = journal_response.json()["o:id"]
            print(f"   ✅ Created Journal [ID={journal_id}]")
        else:
            print(f"   ❌ Failed to create journal: {journal_response.status_code}")
            print(journal_response.text)
            journal_id = None

    if not journal_id:
        print("   Stopping here — cannot proceed without a valid journal.")
        continue

    # -----------------------------------------------------------
    # 2) Ensure the ISSUE exists (attempt to find it first)
    # -----------------------------------------------------------
    print(f"Checking if Issue '{issue}' already exists by identifier='{issue_identifier}'")
    issue_id = get_resource_by_identifier(issue_identifier)
    if issue_id:
        print(f"   ✅ Re-using existing Issue, ID={issue_id}")
    else:
        print(f"   ❌ Not found. Creating new Issue => '{issue}'")
        # We can parse a date from the issue label if desired:
        date_val = extract_date_from_issue_label(issue) or "1900-01-01"

        issue_payload = {
            "o:resource_template": {"o:id": TEMPLATE_ISSUE},
            "o:resource_class": {"o:id": CLASS_ISSUE},
            "o:is_public": True,
            "dcterms:title": [{
                "type": "literal",
                "property_id": PROPERTY_ID_TITLE,
                "@value": issue
            }],
            "dcterms:identifier": [{
                "type": "literal",
                "property_id": PROPERTY_ID_IDENTIFIER,
                "@value": issue_identifier
            }],
            "dcterms:date": [{
                "type": "literal",
                "property_id": PROPERTY_ID_DATE,
                "@value": date_val
            }],
            "dcterms:isPartOf": [{
                "type": "resource",
                "property_id": PROPERTY_ID_IS_PART_OF,
                "value_resource_id": journal_id
            }]
        }

        issue_response = requests.post(
            f"{API_BASE}/items",
            params=params,
            headers=headers,
            json=issue_payload,
            verify=False
        )
        if issue_response.status_code in [200, 201]:
            issue_id = issue_response.json()["o:id"]
            print(f"   ✅ Created Issue [ID={issue_id}]")
        else:
            print(f"   ❌ Failed to create issue: {issue_response.status_code}")
            print(issue_response.text)
            issue_id = None

    if not issue_id:
        print("   Stopping here — cannot proceed without a valid issue.")
        continue

    # -----------------------------------------------------------
    # 3) Create or Update the PAGE
    # -----------------------------------------------------------
    
    jpg_path = Path(f"./Journals Data/{journal}/{volume}/{issue}/{page}.jpg")
    xml_path = Path(f"./XMLs/Habl_1_jahrgang_pagexml/{uid}.xml")
    extracted_text = read_xml_content(xml_path)

    page_data = {
        # Resource Template & Class
        "o:resource_template": {"o:id": TEMPLATE_PAGE},
        "o:resource_class": {"o:id": CLASS_PAGE},
        "o:is_public": True,

        # dcterms:title for the Page
        "dcterms:title": [{
            "type": "literal",
            "property_id": PROPERTY_ID_TITLE,
            "@value": f"{journal} - Issue {issue} - Page {page}"
        }],
        "dcterms:identifier": [{
            "type": "literal",
            "property_id": PROPERTY_ID_IDENTIFIER,
            "@value": page_identifier
        }],

        # Example additional properties
        "dcterms:isPartOf": [{
            "type": "resource",
            "property_id": PROPERTY_ID_IS_PART_OF,
            "value_resource_id": issue_id
        }],

        # If you store the page number, the issue name, or the extracted text
        "foaf:page": [{
            "type": "literal",
            "property_id": 174,
            "@value": page
        }],
        "bibo:issue": [{
            "type": "literal",
            "property_id": 103,
            "@value": issue
        }],
        "extracttext:extracted_text": [{
            "type": "literal",
            "property_id": 185,  
            "@value": extracted_text
        }],

        # Attach the scanned image + OCR file
        "o:media": [
            {"o:ingester": "upload", "file_index": 0, "dcterms:title": [{"@value": "Scan"}]},
            {"o:ingester": "upload", "file_index": 1, "dcterms:title": [{"@value": "OCR"}]}
        ],

        # Just a note
        "dcterms:description": [{
            "type": "literal",
            "property_id": 4,
            "@value": "Created automatically via Omeka API"
        }]
    }

    # POST with multipart form-data to upload files
    files = {
        "data": (None, json.dumps(page_data), "application/json"),
        "file[0]": (jpg_path.name, open(jpg_path, "rb"), "image/jpeg"),
        "file[1]": (xml_path.name, open(xml_path, "rb"), "application/xml")
    }

    print(f"Creating Page => '{page_identifier}'")
    page_response = requests.post(
        f"{API_BASE}/items",
        files=files,
        params=params,
        verify=False
    )

    if page_response.status_code in [200, 201]:
        new_page_id = page_response.json()["o:id"]
        print(f"   ✅ Created Page [ID={new_page_id}] for {page_identifier}")
    else:
        print(f"   ❌ Failed to create page {page_identifier}: {page_response.status_code}")
        print(page_response.text)


In [ ]:
# link journals to issues and issues to pages via HasPart
import requests
import json

# ===== CONFIGURE YOUR API & AUTH =====
API_BASE = "https://persianpress.dh.huji.ac.il/issues/api"

PARAMS  = {"key_identity": KEY_IDENTITY, "key_credential": KEY_CREDENTIAL}
HEADERS = {"Content-Type": "application/json"}

# ===== RESOURCE CLASSES & PROPERTY IDS (adjust if yours differ) =====
CLASS_JOURNAL = 61  # e.g. "Bibliographic Resource"
CLASS_ISSUE   = 60  # e.g. "Periodical Issue"
CLASS_PAGE    = 50  # e.g. "Page"

PROPERTY_ID_IS_PART_OF = 33  # dcterms:isPartOf
PROPERTY_ID_HAS_PART   = 34  # dcterms:hasPart

def get_items_by_resource_class(class_id):
    """
    Returns a list of item dicts for all items 
    that have the given resource_class_id.
    For large sites, you'd implement pagination or multiple calls.
    """
    url = f"{API_BASE}/items"
    params = {
        "resource_class_id": class_id,
        "per_page": 1000,  # or implement pagination if needed
        **PARAMS
    }
    resp = requests.get(url, params=params, verify=False)
    if resp.status_code == 200:
        return resp.json()
    else:
        print(f"❌ Error retrieving items for class_id={class_id}: {resp.status_code} {resp.text}")
        return []

def get_children_by_isPartOf(parent_id):
    """
    Returns all items for which 'dcterms:isPartOf' = <parent_id>.
    That means these items are children of parent_id.
    """
    url = f"{API_BASE}/items"
    params = {
        "property[0][property]": "dcterms:isPartOf",
        "property[0][type]": "res",  # 'res' means resource reference
        "property[0][text]": parent_id,
        "per_page": 1000,
        **PARAMS
    }
    r = requests.get(url, params=params, verify=False)
    if r.status_code == 200:
        return r.json()
    else:
        print(f"❌ Error retrieving children for parent={parent_id}: {r.status_code} {r.text}")
        return []

def get_full_item_data(item_id):
    """
    GET the full JSON for the item, so we can re-PUT or re-PATCH
    without losing other metadata (titles, identifiers, etc.).
    """
    url = f"{API_BASE}/items/{item_id}"
    resp = requests.get(url, params=PARAMS, verify=False)
    if resp.status_code == 200:
        return resp.json()
    else:
        print(f"❌ Error retrieving item_id={item_id}: {resp.status_code} {resp.text}")
        return None

def put_item_data(item_id, item_json):
    """
    PUT the entire JSON to Omeka S for item_id.
    This overwrites the existing item but preserves 
    all fields we leave in item_json.
    We must remove read-only fields before PUT.
    """
    url = f"{API_BASE}/items/{item_id}"
    # Remove read-only or unneeded fields
    for field in [
        "o:id", "o:created", "o:modified", "o:owner",
        "o:resource_type", "o:thumbnail", "o:created",
        "o:media", "o:link", "o:item_set", "o:is_public", 
        "o:modules", "o:site", "o:request", "o:module",
        "@type", "@context", "@id",
    ]:
        item_json.pop(field, None)

    resp = requests.put(url, params=PARAMS, headers=HEADERS, json=item_json, verify=False)
    if resp.status_code in [200, 201]:
        print(f"   ✅ PUT item_id={item_id} succeeded.")
        return True
    else:
        print(f"   ❌ PUT item_id={item_id} failed: {resp.status_code} {resp.text}")
        return False

def safely_append_hasPart(parent_id, child_id):
    """
    1) GET the full item JSON for parent_id
    2) read existing 'dcterms:hasPart'
    3) if child_id not in there, append it
    4) PUT the entire updated item
    """
    parent_data = get_full_item_data(parent_id)
    if not parent_data:
        print(f"   ❌ Could not retrieve parent [ID={parent_id}] to patch hasPart.")
        return

    # 1) read existing dcterms:hasPart
    if "dcterms:hasPart" not in parent_data:
        parent_data["dcterms:hasPart"] = []

    # 'dcterms:hasPart' is typically a list of objects, e.g.:
    # [
    #   {
    #       "type": "resource",
    #       "property_id": 34,
    #       "value_resource_id": 123
    #   },
    #   ...
    # ]
    has_parts = parent_data["dcterms:hasPart"]

    # 2) Check if child is already in hasPart
    already_linked = any(
        (hp.get("value_resource_id") == child_id)
        for hp in has_parts
    )
    if already_linked:
        print(f"   (Skipping) Parent={parent_id} already hasPart => Child={child_id}")
        return

    # 3) If not linked, append a new entry
    new_entry = {
        "type": "resource",
        "property_id": PROPERTY_ID_HAS_PART,
        "value_resource_id": child_id
    }
    has_parts.append(new_entry)

    # 4) PUT the entire updated item back to Omeka
    success = put_item_data(parent_id, parent_data)
    if success:
        print(f"   🔗 Parent [ID={parent_id}] => hasPart => [ID={child_id}] (APPENDED)")

def main():
    # ====================================
    # 1) For each Journal, find its Issues
    # ====================================
    print("=== Linking Journals => Issues (Has Part) ===")
    journals = get_items_by_resource_class(CLASS_JOURNAL)
    print(f"Retrieved {len(journals)} Journals (class_id={CLASS_JOURNAL}).")

    for j_data in journals:
        journal_id = j_data["o:id"]
        print(f"\n--- Journal [ID={journal_id}] ---")
        # find child items that have isPartOf = this journal
        journal_children = get_children_by_isPartOf(journal_id)
        print(f"   Found {len(journal_children)} items referencing this Journal via isPartOf.")
        for child_item in journal_children:
            child_id = child_item["o:id"]
            # Append to "hasPart" if not present
            safely_append_hasPart(journal_id, child_id)

    # ==================================
    # 2) For each Issue, find its Pages
    # ==================================
    print("\n=== Linking Issues => Pages (Has Part) ===")
    issues = get_items_by_resource_class(CLASS_ISSUE)
    print(f"Retrieved {len(issues)} Issues (class_id={CLASS_ISSUE}).")

    for iss_data in issues:
        issue_id = iss_data["o:id"]
        print(f"\n--- Issue [ID={issue_id}] ---")
        # find child items that have isPartOf = this issue
        children = get_children_by_isPartOf(issue_id)
        print(f"   Found {len(children)} items referencing this Issue via isPartOf.")
        for child_item in children:
            child_id = child_item["o:id"]
            safely_append_hasPart(issue_id, child_id)

    print("\nAll done! Your Journals should now have 'Has Part' for their Issues, "
          "and Issues should have 'Has Part' for their Pages, without losing titles.")


if __name__ == "__main__":
    main()


In [ ]:
#associate site
import requests
import json

# ===== CONFIGURE YOUR API & AUTH =====
API_BASE = "https://persianpress.dh.huji.ac.il/issues/api"
KEY_IDENTITY = "L8ZbWWoFDuENrz6PLkJwcqLcgIFoTw2N"
KEY_CREDENTIAL = "9ince7m5EtRAQ7oqosEEDWn1siXROG7e"

PARAMS  = {"key_identity": KEY_IDENTITY, "key_credential": KEY_CREDENTIAL}
HEADERS = {"Content-Type": "application/json"}

# The site ID for "Persian Press Pilot"
SITE_ID = 1  # <-- REPLACE with the actual ID of your site

def get_all_items():
    """
    Returns a list of all items (max 1000).
    If you have more than 1000 items, implement pagination or loop multiple pages.
    You can also filter by resource_class_id, etc., if you only want certain items.
    """
    url = f"{API_BASE}/items"
    params = {
        "per_page": 1000,
        **PARAMS
    }
    resp = requests.get(url, params=params, verify=False)
    if resp.status_code == 200:
        return resp.json()
    else:
        print(f"❌ Error retrieving items: {resp.status_code} {resp.text}")
        return []

def get_item_json(item_id):
    """
    Retrieves the full JSON for the item, 
    so we can re-PUT it without losing metadata.
    """
    url = f"{API_BASE}/items/{item_id}"
    resp = requests.get(url, params=PARAMS, verify=False)
    if resp.status_code == 200:
        return resp.json()
    else:
        print(f"❌ GET item_id={item_id} error: {resp.status_code} {resp.text}")
        return None

def put_item_json(item_id, item_data):
    """
    PUT (overwrite) the entire item with item_data,
    but keep all existing metadata we didn't remove from item_data.
    Must remove read-only fields first.
    """
    url = f"{API_BASE}/items/{item_id}"

    # Remove read-only fields that cause errors or duplication
    for field in [
        "o:id", "o:created", "o:modified", "o:owner",
        "o:resource_type", "o:thumbnail", "o:created",
        "o:media", "o:link", "o:item_set", "o:is_public",
        "o:modules", "o:site", "o:request", "o:module",
        "@type", "@context", "@id",
    ]:
        item_data.pop(field, None)

    resp = requests.put(url, params=PARAMS, headers=HEADERS, json=item_data, verify=False)
    if resp.status_code in [200, 201]:
        print(f"   ✅ PUT item_id={item_id} succeeded.")
        return True
    else:
        print(f"   ❌ PUT item_id={item_id} failed: {resp.status_code} {resp.text}")
        return False

def safely_append_site(item_id, site_id):
    """
    1) GET the full item JSON
    2) Check the 'o:site' array
    3) If site_id not found, append it
    4) PUT the updated item
    """
    data = get_item_json(item_id)
    if not data:
        return

    # "o:site" is a list of site references, e.g.: 
    # [ { "o:id": <siteID>, "@id": "...", "o:slug": "myslug"}, ... ]
    if "o:site" not in data:
        data["o:site"] = []

    # Check if site already in the list
    already_in_site = any(sref.get("o:id") == site_id for sref in data["o:site"])
    if already_in_site:
        print(f"   (Skipping) item_id={item_id} is already in site {site_id}.")
        return

    # Append a minimal site reference to the 'o:site' array
    new_site_ref = {
        "o:id": site_id
    }
    data["o:site"].append(new_site_ref)

    # Now PUT the entire item back
    success = put_item_json(item_id, data)
    if success:
        print(f"   🔗 item_id={item_id} ADDED to site {site_id}.")

def main():
    items = get_all_items()
    print(f"Found {len(items)} total items. Adding them to site_id={SITE_ID} if not present.\n")

    for it in items:
        item_id = it["o:id"]
        print(f"--- Processing item_id={item_id} ---")
        safely_append_site(item_id, SITE_ID)

    print("\nAll done. Items should now appear in site with ID=", SITE_ID)


if __name__ == "__main__":
    main()


In [ ]:
# trigger an action of a module (refresh extract text)

import requests
import math
import json

# ========== CONFIGURE YOUR API / AUTH / CLASSES ===================== #
API_BASE = "https://persianpress.dh.huji.ac.il/issues/api"

PARAMS  = {"key_identity": KEY_IDENTITY, "key_credential": KEY_CREDENTIAL}
HEADERS = {"Content-Type": "application/json"}

# Resource class ID for "Page"
CLASS_PAGE = 50

# We will process re-extractions in smaller chunks to avoid timeouts
CHUNK_SIZE = 50

# ==================================================================== #

def list_all_pages(page_size=200):
    """
    Retrieve ALL items with resource_class_id=CLASS_PAGE (50),
    in chunks of <page_size> from the API. 
    Returns a combined list of items.
    """
    all_items = []
    current_page = 1

    while True:
        query_params = {
            **PARAMS,
            "resource_class_id": CLASS_PAGE,
            "per_page": page_size,
            "page": current_page
        }
        url = f"{API_BASE}/items"
        resp = requests.get(url, params=query_params, verify=False)
        if resp.status_code == 200:
            items = resp.json()
            if not items:
                break  # No more results
            all_items.extend(items)
            print(f"Fetched page={current_page} with {len(items)} items.")
            current_page += 1
        else:
            print(f"❌ Error listing items, status={resp.status_code}")
            print("Response:", resp.text)
            break

    print(f"Done! Fetched a total of {len(all_items)} 'Page' items.")
    return all_items

def refresh_extract_text_for_item(item_id):
    """
    Sends a PARTIAL PATCH that sets "extract_text_action=refresh",
    triggering re-extraction for the item.
    """
    patch_url = f"{API_BASE}/items/{item_id}"
    patch_data = {
        # According to your module, "refresh" re-runs text extraction if files are local
        "extract_text_action": "refresh"
    }
    resp = requests.patch(patch_url, params=PARAMS, headers=HEADERS, json=patch_data, verify=False)
    if resp.status_code in [200, 204]:
        print(f"   ✅ Re-extraction triggered for item_id={item_id}")
        return True
    else:
        print(f"   ❌ Failed to patch item_id={item_id}: {resp.status_code}")
        print(resp.text)
        return False

def main():
    # 1) List all Page items, in smaller page chunks (200 each)
    all_pages = list_all_pages(page_size=200)
    print(f"Total Pages retrieved: {len(all_pages)}")

    if not all_pages:
        print("No pages found, nothing to refresh.")
        return

    # 2) Extract IDs
    all_page_ids = [p["o:id"] for p in all_pages]

    # 3) Chunk them in groups of CHUNK_SIZE to avoid timeouts in consecutive patches
    for i in range(0, len(all_page_ids), CHUNK_SIZE):
        subset = all_page_ids[i:i + CHUNK_SIZE]
        print(f"\n=== Processing items {i+1} through {i+len(subset)} of {len(all_page_ids)} ===")
        for item_id in subset:
            refresh_extract_text_for_item(item_id)

    print("\nAll done! Each item has been requested to 'refresh' extract text. "
          "Check your Omeka Jobs or logs to see them re-run, and confirm new text is extracted.")

if __name__ == "__main__":
    main()
